#### Fetch Stock Posts

In [1]:
# Get stock name
stock_symbol = "AAPL"

In [8]:
# Reddit posts fetching

## Import libraries 
import praw
import os
from dotenv import load_dotenv

load_dotenv()

## Setup
reddit = praw.Reddit(
    client_id=os.getenv("reddit_client_id"),
    client_secret=os.getenv("reddit_client_secret"),
    user_agent=os.getenv("reddit_user_agent")
)

def fetch_posts(stock_symbol):
  ### subreddit
  subreddit = reddit.subreddit("stocks")

  ### search query
  query = f'title:{stock_symbol}'

  ### fetch posts
  results = subreddit.search(query, limit=10, sort='top')

  return results

## Call function
apple_posts = fetch_posts(stock_symbol)

## Print first post
first = next(apple_posts)
print(first.title)

I have $213,000 in Apple (AAPL) stock, should I start to diversify?


#### Text Preparation

In [9]:
## Import libraries
import re

## Concatenate body with title
text = f"{first.title}\n{first.selftext}"

## Clean text 
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  ### remove links
    text = re.sub(r'\s+', ' ', text)     ### collapse whitespace
    text = re.sub(r'[^\w\s.,!?\'"]', '', text)  ### remove unwanted chars
    return text.strip()

cleaned_text = clean_text(text)
print(cleaned_text)

I have 213,000 in Apple AAPL stock, should I start to diversify? I always tell myself to stop buying Apple, but I always give in because Apple is such a cash machine. Would you start diversifying new money in companies such as Square SQ, Microsoft MSFT and Amazon AMZN. Or would you keep adding more to Apple AAPL? Edit Apple AAPL is my entire portfolio, don't own anything else. Second Edit Damn this blew up.


#### Text Embedding

In [11]:
## Import libraries
from sentence_transformers import SentenceTransformer

## Generate embeddings
embedding_model = SentenceTransformer("yiyanghkust/finbert-tone")
embedding = embedding_model.encode(text)

print(embedding.shape)

No sentence-transformers model found with name yiyanghkust/finbert-tone. Creating a new one with mean pooling.


(768,)


#### Model Prediction

In [24]:
import tensorflow as tf

## Reshape embeddings 
print(f"Original embedding shape: {embedding.shape}")
embedding_reshaped = embedding.reshape(1, -1)
print(f"Dimensions: {embedding_reshaped.shape}")


## Load prediction
model = tf.keras.models.load_model("model/models/model_v1.keras")

## Perform Prediction
prediction = model.predict(embedding_reshaped)
classes = ["Negative", "Neutral", "Positive"]
print(f"Prediction: {prediction}")
print(f"class Index {prediction.argmax(-1)}")
print(f"Class: {classes[prediction.argmax(-1)[0]]}")

Original embedding shape: (768,)
Dimensions: (1, 768)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Prediction: [[0.07035457 0.9028228  0.02682256]]
class Index [1]
Class: Neutral
